In [ ]:
"""
Window size: 1, 7, 30 day -> HDBSCAN
min_samples = 10, 50, 200
데이터가 모자라는 경우 이전날짜의 데이터 편입
json 저장, 형식:
    {
        {
            "Day": YYYYMMDD (20191231인 경우(Window size == 30), 20191202 - 20191231의 데이터를 모은 것을 의미),
            "Cluster_{index}": [IDs],
            "Cluster_{index}": [IDs],
            ...
            "Center_Link": [Cluster_01의 대표 Link(Centroid와 가장 가까운 embedding의 Link), Cluster_02의 대표 Link],
            "Noise": [IDs]
        },
        ...
    }
"""

In [19]:
import numpy as np
from datetime import datetime, timedelta

import hdbscan
import json
import os

embedding_path = "../Data/News/Embeddings"
file_dirs = [os.path.join(embedding_path, file) for file in os.listdir(embedding_path) if file.endswith('.json')]

In [25]:
ref_N = 30
with open(f"reference_{ref_N}.json") as f:
    reference = json.load(f)

tmp = None
ref_dict = {}
for row in reference:
    key = list(row.keys())[0]
    file_value = row[key][-1]

    if tmp is not None and file_value != tmp:
        date_obj = datetime.strptime(key, "%Y%m%d") - timedelta(days=1)
        # ref_dict[date_obj] = tmp
        ref_dict[date_obj.strftime("%Y%m%d")] = tmp

    tmp = file_value

date_list = [list(row.keys())[0] for row in reference]

In [26]:
ref_dict

{'20190630': '005930_2019_Q1.json',
 '20200129': '005930_2019_Q2.json',
 '20200630': '005930_2020_Q1.json',
 '20210129': '005930_2020_Q2.json',
 '20210729': '005930_2021_Q1.json',
 '20211231': '005930_2021_Q2.json',
 '20220630': '005930_2022_Q1.json',
 '20230129': '005930_2022_Q2.json',
 '20230630': '005930_2023_Q1.json',
 '20231231': '005930_2023_Q2.json',
 '20240630': '005930_2024_Q1.json',
 '20250129': '005930_2024_Q2.json'}

In [ ]:
def load_embeddings(date, reference_data):
    date = datetime.strptime(date, format="%Y%m%d")

    for base_date in reference_data.keys():
        if date <= datetime.strptime(base_date, format="%Y%m%d"):
            load_file = reference_data[base_date]
            break
    
    return load_file

def HDBSCAN(
        embeddings,
        metric: str = 'cosine',
        cluster_selection_method: str = "eom",
        min_cluster_size = None
    ):
    if min_cluster_size:
        min_cluster_size = embeddings.shape[0] // 10
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        metric=metric,
        cluster_selection_method=cluster_selection_method
    )

    clusterer.fit(embeddings)
    labels = clusterer.labels_

    return labels

In [ ]:
"""
IDEAs:
    - 날짜별로 30일간의 embedding 벡터에 대해서 Cluster, Representative vector까지 구하는 것은 good.
    - New cluster의 등장을 어떻게 표현하지?
        => Representative vector들만 모아서, 그 벡터들의 alignment를 생각해보는 것은? => 그렇다면 기준이 있어야 하지
            + t 시점의 represnetative vector과 t+1시점의 벡터가 서로 같다 / 다르다는 어떻게 하는가? : 당연히 metric 기반이겠지.
            + 그러면 같다 다르다를 정하는 metric을 내가 임의로 저장해도 괜찮은가? X, 내가 실제로 저장하기도 어려워보이고...
            + 그렇다면 t시점에 만들어진 cluster N개, t+1시점에 만들어진 cluster M개, 서로 vector간 distance metric 비교하면 N*M matrix
            + Get minimum by row -> i-th(in time==t) cluster <~> j-th(in time==t+1) cluster Linked
            + N > M인 경우, (N-M)개의 cluster: depreciated (한번에 사라지면 좀 그렇지? 그래서 patience를 도입하자)
            + M > N인 경우, (M-N)개의 cluster: newly created
            + Minimum distance와 patience를 어떻게 구할 수 있을지??
            + patience = 3days >> 이정도가 지나도 distance가 threshold밑으로 내려가지 않으면 클러스터가 사라진 것으로 판단
"""